# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**

* **Nombres y matrículas:**

  *   Carlos Pano Hernandez - A01066264
  *   Elemento de lista
  *   Elemento de lista

* **Número de Equipo:**


* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [13]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

import os
import requests
import librosa

# Define the download directory
download_dir = "data"

# Create directory if it doesn't exist
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# URLs for Aesop's Fables audio files
audio_urls = {
    "audio_1": "https://www.gutenberg.org/files/21144/mp3/21144-01.mp3",
    "audio_4": "https://www.gutenberg.org/files/21144/mp3/21144-04.mp3", 
    "audio_5": "https://www.gutenberg.org/files/21144/mp3/21144-05.mp3",
    "audio_6": "https://www.gutenberg.org/files/21144/mp3/21144-06.mp3",
    "audio_14": "https://www.gutenberg.org/files/21144/mp3/21144-14.mp3",
    "audio_22": "https://www.gutenberg.org/files/21144/mp3/21144-22.mp3",
    "audio_24": "https://www.gutenberg.org/files/21144/mp3/21144-24.mp3",
    "audio_25": "https://www.gutenberg.org/files/21144/mp3/21144-25.mp3",
    "audio_26": "https://www.gutenberg.org/files/21144/mp3/21144-26.mp3",
    "audio_27": "https://www.gutenberg.org/files/21144/mp3/21144-27.mp3"
}

# Download each audio file
for audio_name, url in audio_urls.items():
    file_path = os.path.join(download_dir, f"{audio_name}.mp3")
    
    # Only download if file doesn't exist
    if not os.path.exists(file_path):
        response = requests.get(url)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded {audio_name}")
        else:
            print(f"Failed to download {audio_name}")
    else:
        print(f"{audio_name} already exists")
        
    # Load and print duration for each audio file
    audio, sample_rate = librosa.load(file_path)
    print(f"Duración de {audio_name}: {len(audio)/sample_rate:.1f} segundos")


audio_1 already exists
Duración de audio_1: 72.6 segundos
audio_4 already exists
Duración de audio_4: 67.1 segundos
audio_5 already exists
Duración de audio_5: 58.4 segundos
audio_6 already exists
Duración de audio_6: 71.8 segundos
audio_14 already exists
Duración de audio_14: 58.1 segundos
audio_22 already exists
Duración de audio_22: 57.2 segundos
audio_24 already exists
Duración de audio_24: 64.5 segundos
audio_25 already exists
Duración de audio_25: 56.5 segundos
audio_26 already exists
Duración de audio_26: 63.7 segundos
audio_27 already exists
Duración de audio_27: 60.3 segundos


# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

In [2]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [3]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

In [4]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 4:**

In [5]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

In [6]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 6:**

* #### **Incluyan sus conclusiones de la actividad audio-a-texto:**



None

# **Fin de la actividad LDA y LMM: audio-a-texto**